# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(14)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,277.73,99,100,2.90,GS,1698622805
1,1,port elizabeth,-33.9180,25.5701,287.31,79,20,5.66,ZA,1698622829
2,2,constantia,44.1833,28.6500,285.53,72,0,3.49,RO,1698622842
3,3,lahad datu,5.0268,118.3270,300.23,81,92,0.29,MY,1698623060
4,4,moree,-29.4667,149.8500,300.30,30,0,6.98,AU,1698623060
5,5,cidade velha,14.9167,-23.6167,298.41,73,20,3.60,CV,1698623061
6,6,remire-montjoly,4.9167,-52.2667,301.17,83,0,2.06,GF,1698622869
7,7,antofagasta,-23.6500,-70.4000,290.75,75,17,2.68,CL,1698623061
8,8,bairnsdale,-37.8333,147.6167,295.40,44,100,2.24,AU,1698623061
9,9,jamestown,42.0970,-79.2353,282.05,96,100,2.06,US,1698622925


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
%%capture --no-display

map_plot_1 = city_data_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    tiles = "OSM"
)



# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lat,Lng]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
good_cities_df = city_data_df[(city_data_df["Max Temp"] <= 300)& (city_data_df["Max Temp"] >= 274) &(city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] <= 4.5)]

good_cities_df.dropna()

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
good_cities_df.head(11)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,constantia,44.1833,28.6500,285.53,72,0,3.49,RO,1698622842
26,26,zahedan,29.4963,60.8629,286.10,58,0,2.06,IR,1698622803
40,40,awjilah,29.1081,21.2869,294.52,46,0,3.11,LY,1698623069
42,42,murzuq,25.9155,13.9184,295.66,26,0,3.90,LY,1698623069
64,64,adana,36.9850,35.2881,291.16,72,0,1.54,TR,1698623074
103,103,mawu,29.5923,107.3139,287.99,88,0,0.78,CN,1698623084
111,111,keflavik,64.0049,-22.5624,277.39,70,0,2.06,IS,1698622933
128,128,laguna,38.4210,-121.4238,297.31,19,0,2.06,US,1698622927
167,167,kalinagar,28.6167,80.1000,290.92,66,0,0.68,IN,1698623097
168,168,argo,37.6333,22.7333,292.35,82,0,1.34,GR,1698623047


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = good_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(9)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,constantia,44.1833,28.6500,285.53,72,0,3.49,RO,1698622842,
26,26,zahedan,29.4963,60.8629,286.10,58,0,2.06,IR,1698622803,
40,40,awjilah,29.1081,21.2869,294.52,46,0,3.11,LY,1698623069,
42,42,murzuq,25.9155,13.9184,295.66,26,0,3.90,LY,1698623069,
64,64,adana,36.9850,35.2881,291.16,72,0,1.54,TR,1698623074,
103,103,mawu,29.5923,107.3139,287.99,88,0,0.78,CN,1698623084,
111,111,keflavik,64.0049,-22.5624,277.39,70,0,2.06,IS,1698622933,
128,128,laguna,38.4210,-121.4238,297.31,19,0,2.06,US,1698622927,
167,167,kalinagar,28.6167,80.1000,290.92,66,0,0.68,IN,1698623097,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(6)
#hotel_df

Starting hotel search
constantia - nearest hotel: Hotel Chérica
zahedan - nearest hotel: هتل صالح;بانک ملی
awjilah - nearest hotel: No hotel found
murzuq - nearest hotel: فندق باريس - مرزق
adana - nearest hotel: Le Grand
mawu - nearest hotel: No hotel found
keflavik - nearest hotel: Núpan Deluxe
laguna - nearest hotel: Holiday Inn Express & Suites
kalinagar - nearest hotel: No hotel found
argo - nearest hotel: Apollon Hotel
tazacorte - nearest hotel: App Leyma
woodlake - nearest hotel: Sleep Inn
zara - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
mengcheng chengguanzhen - nearest hotel: No hotel found
wuwei - nearest hotel: Hepingli Boutique Hotel
newman - nearest hotel: No hotel found
oranjemund - nearest hotel: No hotel found
prince george - nearest hotel: Travelodge by Wyndham Prince George
harper - nearest hotel: No hotel found
maun - nearest hotel: Center Lodge Conference Center
quesnel - nearest hotel: Quality Inn
shingu - nearest hotel: ホテルニューパレス
al hoce

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,constantia,44.1833,28.6500,285.53,72,0,3.49,RO,1698622842,Hotel Chérica
26,26,zahedan,29.4963,60.8629,286.10,58,0,2.06,IR,1698622803,هتل صالح;بانک ملی
40,40,awjilah,29.1081,21.2869,294.52,46,0,3.11,LY,1698623069,No hotel found
42,42,murzuq,25.9155,13.9184,295.66,26,0,3.90,LY,1698623069,فندق باريس - مرزق
64,64,adana,36.9850,35.2881,291.16,72,0,1.54,TR,1698623074,Le Grand
103,103,mawu,29.5923,107.3139,287.99,88,0,0.78,CN,1698623084,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

map_plot_2 = hotel_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    tiles = "OSM",
    hover_cols = ['Hotel Name', 'Country']
)



# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lat,Lng]   (Hotel Name,Country)